In [ ]:
import os
print(f"当前目录: {os.getcwd()}")
if 'arch-gaussian' in os.listdir():
    JUPYTER_ROOT = os.getcwd()
    os.chdir('arch-gaussian')
    print(f"更改后的目录: {os.getcwd()}")
#针对远程jupyter目录的操作， 本地运行可以无视
if 'scripts' in os.getcwd():
    os.chdir("..")
    print(f"更改后的目录: {os.getcwd()}")


In [ ]:
import config

# 基础设置
config.project_root = r"D:\M.Arch\MastersThesis\ArchGaussian-Project\shared"
config.scene_name = r"Nanjing\Jimingsi" # 输入文件夹名称
config.output_name = r"Nanjing\Jimingsi"  # 输出文件夹名称
config.sh_degree = 3  # 0~3， 改为0可以缩小文件大小，但是无法被现有的unity工具识别， 推荐使用默认3
config.epochs = 1000  # 训练轮次， 3000， 7000， 15000， 30000
config.loaded_iter = None # 是否加载已经训练好的轮次，None 或数字
# 高级设置
config.resolution = 1 # 是否压缩图像，1为原尺寸，-1为自动压缩到不大于1600像素， 支持2的次方的数字
config.densify_until_iter = 15000  # 致密化结束的轮次
config.densify_grad_threshold = 0.0002  # 致密化的阈值，越小增加的越快

# 配置结束，更新配置文件
config.update_colmap_args()
config.update_args()

# 打印修改过后的参数
config.print_updated_colmap_args()
config.print_updated_args()

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("./src")
import os
import torch
import numpy as np
np.set_printoptions(suppress=True)
from config import args  # 正式导入args


In [ ]:
print(sys.path)

In [ ]:
# 创建scene info
from manager.scene_manager import load_and_fix_scene

# 根据相机姿态，自动估算地面向上向量，并将场景
scene_manager = load_and_fix_scene(args)


In [ ]:
from manager.camera_manager import CameraManager
cm = CameraManager()
train_cameras, test_cameras = cm.create_cameras(args, scene_manager.scene_info)

In [ ]:
from manager.gaussian_manager import GaussianManager
gm = GaussianManager(args, scene_manager.scene_info)
print(f"num points: {gm.gaussians.get_xyz.shape[0]}")

In [ ]:
from manager.train_manager import init_snapshot, take_snapshot, SnapshotCameraMode, SnapshotFilenameMode
init_snapshot(0)
def post_socket(**kwargs):
    """完成每一轮训练后的后处理内容"""
    _iteration = kwargs['iteration']
    take_snapshot(cm, gm,
                  _camera_mode=SnapshotCameraMode.SLOW_ROTATE,
                  _slow_ratio=5,
                  _filename_mode=SnapshotFilenameMode.BY_SNAP_COUNT,
                  _folder_name="snapshots",
                  _iteration_gap=100,
                  _first_period_iteration_gap=10,
                  _first_period_end=400,
                  **kwargs)
    _gaussians = kwargs['gaussians']
    if _iteration % 1000 == 0:
        print(_gaussians.get_xyz.shape)

In [ ]:
from manager.train_manager import train, init_output_folder

init_output_folder(args, scene_manager.scene_info)
train(args,scene_manager.scene_info, gm.gaussians, cm.train_cameras, post_socket=post_socket)

In [ ]:
from manager.gaussian_manager import GaussianManager
gm = GaussianManager(args, gm.gaussians) # 从gaussians创建GaussianManager
cam = cm.pick_camera(0)
image = gm.render(cam, convert_to_pil=True)
image